In [1]:
import os
import time
import pickle
from collections import deque

import gym
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt

from agent import A2CAgent

In [2]:
RANDOM_SEED = 123
BUFFER_SIZE = 15
STEP_SIZE = 10
SAVED_DATA_DIR = './saved_data'

In [ ]:
env = gym.make('LunarLander-v2')
env.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)


STATE_SIZE = 8
ACTION_SIZE = 4


agent = A2CAgent(
    state_size=BUFFER_SIZE * STATE_SIZE, 
    action_size=ACTION_SIZE, 
    step_size=STEP_SIZE
)


history = []
start = time.time()

for episode in range(1, 3001):
    score = 0
    step = 0
    buffer = deque(np.zeros((BUFFER_SIZE, STATE_SIZE), dtype='float32'), maxlen=BUFFER_SIZE)
    
    done = False
    state = env.reset()
    buffer.append(state)
    state = np.array(buffer).flatten()
    
    while not done:
        action = agent.get_action(state)
        next_state, reward, done, _ = env.step(action)
        buffer.append(next_state)
        next_state = np.array(buffer).flatten()
        score += reward
        step += 1
        agent.train(state, action, reward, next_state, done)
        state = next_state
    
    elapsed = int(time.time() - start)
    history.append([elapsed, episode, score, step])
    
    if episode % 10 == 0:
        with open(os.path.join(SAVED_DATA_DIR, 'history.pkl'), 'wb') as f:
            pickle.dump(history, f)
        score_mean = np.array(history)[-10:, 2].mean()
        step_mean = np.array(history)[-10:, 3].mean()
        score_total = np.array(history)[:, 2].sum()
        print('episode: {:4d} | score_mean: {:4.2f} | step_mean: {:4.2f} | score_total: {} | elapsed: {}'.format(
            episode, score_mean, step_mean, score_total, elapsed
        ))
    
    if episode % 100 == 0:
        agent.actor_model.save_weights(os.path.join(SAVED_DATA_DIR, f'model/actor-{episode}'))
        agent.critic_model.save_weights(os.path.join(SAVED_DATA_DIR, f'model/critic-{episode}'))
